缓慢降温没用，增加探索性没用，探索性破破奖励信号。

In [1]:
import numpy as np
import keras as k
import rules
import arena as arn
import bot_base2
import bot_bootstrap

ARENA = arn.ARENA
bot_rd = arn.bot_rd
bot_me = arn.bot_me


In [8]:
def bot_mc_f(basebot, verbos=2, RECORD=True, MAXWIDTH=3, NSIMS_FACTOR=50, NSIMS_MAX=1000):
    return mc.BOT(basebot=basebot, verbos=verbos, RECORD=RECORD, MAXWIDTH=MAXWIDTH, NSIMS_FACTOR=NSIMS_FACTOR, NSIMS_MAX=NSIMS_MAX)

In [2]:
# model = k.models.load_model("model_base_allrecords/m{}.keras".format(17544))
# model = k.models.load_model("model_base2/m{}.keras".format(43800))

def bot1_f(verbos=0):
    model = k.models.load_model("model_bootstrap/m{}.keras".format(3100))
    return bot_bootstrap.BOT(model, verbos=verbos)

def bot2_f(verbos=0):
    model = k.models.load_model("model_base2/m{}.keras".format(3100))
    return bot_base2.BOT(model, verbos=verbos)

def bot_rd_f():
    return bot_rd

In [3]:
def ft(N, bot1_f, bot2_f):
    s1, s2 = 0, 0
    for i in range(N):
        cards = rules.CARDS.copy()
        np.random.shuffle(cards)
        
        arena = ARENA(verbos=0, cards=cards.copy())
        arena.registerbot([bot1_f(), bot2_f(), bot2_f()])
        arena.wholegame()
        s1 += (arena.winner == 0)
        
        arena2 = ARENA(verbos=0, cards=cards.copy())
        arena2.registerbot([bot2_f(), bot1_f(), bot1_f()])
        arena2.wholegame()
        s2 += (arena2.winner == 0)
        
    # for i in range(N):
    #     arena = ARENA(verbos=0)
    #     arena.registerbot([BOT(model_0), BOT(model), BOT(model)])
    #     # arena.registerbot([BOT(model_0), bot_rd, bot_rd])
    #     arena.wholegame()
    #     s2 += (arena.winner == 0)
    # return s1
    return s1, s2

In [51]:
ft(100, bot1_f, bot2_f)

(58, 45)

In [4]:
arena = arn.ARENA(1, True)
arena.registerbot([bot1_f(1), bot1_f(1), bot1_f(1)])
arena.wholegame()

0.08259371 4 5 6 7 8
0.05687173 3 4 5 6 7 8
0.055505957 4 5 5 5 5 8
0.026524197 4 5 5 5 5 6
0.009569192 4 5 6 7 8 9
 0 :  地主 : 4 5 6 7 8 | 3 3 5 5 5 7 9 9 J Q Q K K A 小王
0.037723072 7 8 9 10 J
0.00284836 8 9 10 J Q
-0.0075445455 6 7 8 9 10
-0.19617267 
 1 : 农民1 : 7 8 9 10 J | 3 4 4 6 6 8 Q Q A 2 2 大王
0.14403152 
 2 : 农民2 :  | 3 4 6 7 8 9 10 10 10 J J K K A A 2 2
-0.15247811 
 3 :  地主 :  | 3 3 5 5 5 7 9 9 J Q Q K K A 小王
0.24310108 8
0.237066 3
0.22092527 4 4
0.19132127 Q Q
0.18825415 6 6
 4 : 农民1 : 8 | 3 4 4 6 6 Q Q A 2 2 大王
0.3700298 A
0.365588 K
0.3580296 2
0.35585812 J
0.33816642 9
 5 : 农民2 : A | 3 4 6 7 8 9 10 10 10 J J K K A 2 2
-0.29047853 
-0.38736016 小王
 6 :  地主 :  | 3 3 5 5 5 7 9 9 J Q Q K K A 小王
0.2868661 2
0.27954176 
0.19423851 大王
 7 : 农民1 : 2 | 3 4 4 6 6 Q Q A 2 大王
0.3913065 
 8 : 农民2 :  | 3 4 6 7 8 9 10 10 10 J J K K A 2 2
-0.27146053 
-0.31935227 小王
 9 :  地主 :  | 3 3 5 5 5 7 9 9 J Q Q K K A 小王
0.3466744 3
0.32605356 4 4
0.29814467 6 6
0.29058462 Q Q
0.24756962 2
10 : 农民1 

In [6]:
import re

def repl(matched):
    d = {"11": "J", "12": "Q", "13": "K", "14": "A", "15": "2", "16": "小王", "17": "大王"}
    return d[matched[0]]

def arenabyscratch(l):
    cards = [rules.str2vec(re.sub(r'1[1-7]', repl, v)) for v in re.split(",", l)]
        
    arena=object.__new__(arn.ARENA)
    arena.verbos = 1
    arena.init = np.array(cards, int)
    arena.remain = arena.init.copy()
    arena.lastplay = np.zeros((3, 15), int)
    arena.pos = 0
    arena.b1 = 2
    arena.b2 = 1
    arena.round = 0
    arena.bot = []
    arena.gameover = False
    arena.winner = None
    arena.getdata()
    arena.RECORD = True
    arena.records = []
    if arena.RECORD:
        arena.records.append(arena.copy())
    return arena

arena = arenabyscratch("3 4 4 4 4 5 5 7 8 9 9 9 10 11 12 13 13 14 14 17, 3 5 5 6 6 6 7 7 7 8 9 10 11 12 13 13 15, 3 3 6 8 8 10 10 11 11 12 12 14 14 15 15 15 16")

In [31]:
a1 = arena.records[0].copy(1, True)
a1.registerbot([bot1_f(1), bot1_f(1), bot1_f(1)])
# a1.wholegame()

In [27]:
a1 = arena.records[0].copy(1, True)
bot = bot_conv.BOT(bot_conv.BOT.createmodel())
a1.registerbot([bot, bot, bot])

In [28]:
a1.bot

In [30]:
bot.showChoices(a1)

AttributeError: 'BOT' object has no attribute 'arena'

In [15]:
a1 = arena.records[0].copy(0, True)
a1.update(a1.getChoices()[0])

In [44]:
def createmodel():
    def residual_block(x, filters):
        # Shortcut connection
        shortcut = x
        
        # First convolution
        x = k.layers.Conv2D(filters, (3, 3), padding='same')(x)
        x = k.layers.BatchNormalization()(x)
        x = k.layers.ReLU()(x)
        
        # Second convolution
        x = k.layers.Conv2D(filters, (3, 3), padding='same')(x)
        x = k.layers.BatchNormalization()(x)
        
        # Add shortcut to the output
        x = k.layers.add([x, shortcut])
        x = k.layers.ReLU()(x)
        
        return x

    inputs = k.layers.Input(shape=(15, 15, 9, ))

    # Initial Conv Layer
    x = k.layers.Conv2D(64, (3, 3), padding='same')(inputs)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.ReLU()(x)

    # Residual Blocks
    for _ in range(3):  # You can increase the number of blocks
        x = residual_block(x, 64)

    # Pooling layer
    x = k.layers.GlobalAveragePooling2D()(x)
    
    # Output layer
    outputs = k.layers.Dense(10, activation='softmax')(x)

    model = k.models.Model(inputs, outputs)
    return model

In [45]:
model = createmodel()

In [46]:
model.summary()

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_14      │ (None, 15, 15, 9) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_42 (Conv2D)  │ (None, 15, 15,    │      5,248 │ input_layer_14[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 15, 15,    │        256 │ conv2d_42[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_46 (ReLU)     │ (None, 15, 15,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_43 (Conv2D)  │ (None, 15, 15,    │     36,928 │ re_lu_46[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 15, 15,    │        256 │ conv2d_43[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_47 (ReLU)     │ (None, 15, 15,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_44 (Conv2D)  │ (None, 15, 15,    │     36,928 │ re_lu_47[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 15, 15,    │        256 │ conv2d_44[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 15, 15,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │ re_lu_46[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_48 (ReLU)     │ (None, 15, 15,    │          0 │ add_3[0][0]       │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_45 (Conv2D)  │ (None, 15, 15,    │     36,928 │ re_lu_48[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 15, 15,    │        256 │ conv2d_45[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_49 (ReLU)     │ (None, 15, 15,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_46 (Conv2D)  │ (None, 15, 15,    │     36,928 │ re_lu_49[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 15, 15,    │        256 │ conv2d_46[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (None, 15, 15,    │          0 │ batch_normalizat

 Total params: 229,258 (895.54 KB)

 Trainable params: 228,362 (892.04 KB)

 Non-trainable params: 896 (3.50 KB)

In [19]:
xs = np.random.normal(0, 1, (100, 3, 18, 9))

In [23]:
%%timeit
model(xs)

30.5 ms ± 1.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
%%timeit
model(xs)

16.2 ms ± 461 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
